In [1]:

%load_ext autoreload
%autoreload 2
import random
import numpy as np
import itertools
from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression

In [2]:
indices_used = list(itertools.product(itertools.permutations(range(4), 2), itertools.permutations(range(3), 2), itertools.permutations(range(2), 2)))
operations_used = list(itertools.product(range(4), repeat=3)) # 4 operations, using 3 at a time
operations = [
    lambda x, y: x + y,
    lambda x, y: x - y,
    lambda x, y: x * y,
    lambda x, y: x / y,
]
operation_names = ["+", "-", "*", "/"]
combos = list(itertools.product(indices_used, operations_used))

In [3]:
def get_questions(n):
    res = []
    for i in range(n):
        items = [random.randint(1, 10) for _ in range(4)]
        # What final number can we get?def solve(question):
        # pair up two numbers 1-4, then 1-3, then 1-2.
        # For each pair, we can add, subtract, multiply or divide.
        targets = set()
        for index_order, operation_order in itertools.product(indices_used, operations_used):
            numbers = items.copy()
            valid = True
            for (index1, index2), operation_index in zip(index_order, operation_order):
                val1 = numbers[index1]
                val2 = numbers[index2]
                numbers.pop(max(index1, index2))
                numbers.pop(min(index1, index2))
                if operation_index == 3 and val2 == 0:
                    valid = False
                    break
                numbers.append(operations[operation_index](val1, val2))
            if valid and numbers[0] % 1 == 0 and numbers[0] > 0:
                targets.add(numbers[0])
        target = random.choice(list(targets))
        res.append((items, target))
    return res
create_data = False
if create_data:
    questions = get_questions(10000)
    with open("questions1.txt", "w") as file:
        file.write("\n".join([f"{' '.join([str(item) for item in items])} {int(target)}" for items, target in questions]))
else:
    with open("questions.txt", "r") as file:
        questions = file.read().split("\n")
        questions = [(list(map(int, question.split()[:-1])), int(question.split()[-1])) for question in questions]
print(questions)

[([6, 10, 6, 9], 264), ([8, 9, 8, 7], 632), ([7, 4, 6, 5], 1), ([10, 5, 7, 6], 260), ([3, 10, 2, 8], 190), ([6, 8, 7, 6], 196), ([1, 1, 8, 10], 90), ([2, 9, 4, 8], 72), ([3, 7, 2, 7], 4), ([2, 10, 3, 10], 14), ([2, 9, 1, 4], 47), ([3, 9, 4, 2], 126), ([1, 3, 7, 1], 7), ([8, 2, 10, 6], 168), ([8, 5, 4, 1], 18), ([4, 6, 8, 4], 28), ([10, 2, 9, 2], 16), ([7, 2, 3, 8], 38), ([2, 1, 4, 10], 59), ([6, 10, 3, 6], 5), ([10, 4, 3, 2], 2), ([7, 7, 9, 6], 103), ([10, 1, 6, 7], 9), ([1, 1, 7, 9], 2), ([5, 5, 1, 3], 20), ([1, 1, 3, 6], 28), ([5, 1, 1, 1], 15), ([9, 1, 8, 9], 71), ([10, 10, 8, 3], 776), ([10, 2, 5, 9], 270), ([2, 4, 8, 5], 170), ([1, 1, 10, 3], 9), ([7, 8, 1, 1], 63), ([9, 7, 5, 6], 77), ([8, 3, 7, 6], 71), ([10, 10, 2, 8], 12), ([7, 7, 8, 1], 2), ([7, 3, 1, 4], 36), ([3, 3, 9, 10], 13), ([6, 3, 7, 9], 79), ([2, 5, 3, 4], 22), ([1, 8, 3, 9], 104), ([8, 3, 6, 7], 294), ([5, 7, 5, 10], 36), ([5, 1, 9, 2], 56), ([2, 4, 8, 10], 80), ([1, 10, 10, 8], 92), ([7, 8, 3, 6], 49), ([6, 10, 6, 

In [4]:
use_random = True
def solve(question):
    # pair up two numbers 1-4, then 1-3, then 1-2.
    # For each pair, we can add, subtract, multiply or divide.
    target = question[1]
    i = 0
    while use_random or i < len(combos):
        index_order, operation_order = random.choice(combos) if use_random else combos[i]
        i += 1
        numbers = question[0].copy()
        valid = True
        for (index1, index2), operation_index in zip(index_order, operation_order):
            val1 = numbers[index1]
            val2 = numbers[index2]
            numbers.pop(max(index1, index2))
            numbers.pop(min(index1, index2))
            if operation_index == 3 and val2 == 0:
                valid = False
                break
            numbers.append(operations[operation_index](val1, val2))
        if valid and numbers[0] == target:
            return True, index_order, operation_order, i
    return False, None, None, -1
def print_solution(numbers, index_order, operation_order, answer):
    terms = [str(x) for x in numbers]
    for (index1, index2), operation_index in zip(index_order, operation_order):
        term1 = terms[index1]
        term2 = terms[index2]
        terms.pop(max(index1, index2))
        terms.pop(min(index1, index2))
        new_term = f"({term1} {operation_names[operation_index]} {term2})"
        terms.append(new_term)
    print(" ".join(terms), "=", answer)
debug = False
avg_count = 0
n_entries = 0
solutions = []
for question in tqdm(questions):
    valid, index_order, operation_order, i = solve(question)
    if not valid:
        print("Failed to solve", question)
        break
    else:
        avg_count += i
        n_entries += 1
        solutions.append([question[0], question[1], index_order, operation_order])
        if debug:
            print("input", question, "indices", index_order, "operations", operation_order, "count", i)
            print_solution(question[0], index_order, operation_order, question[1])
            print()

print("Average count", avg_count / n_entries) # 1186.1122 in order, 639.8498 random

  0%|          | 0/10000 [00:00<?, ?it/s]

Average count 645.4637


In [5]:
label_is_3d = True
print(solutions[0])
X = np.zeros((len(solutions), 5))
y = np.zeros((len(solutions), 3, 4)) if label_is_3d else np.zeros((len(solutions), 3*4)) # predict the 3 operations using one hot encoding
for i, sol in enumerate(solutions):
    X[i] = sol[0] + [sol[1]] # input + target
    # y[i] = sol[3] # regression output
    # one hot encoding
    for rule_index in range(3):
        if label_is_3d:
            y[i, rule_index, sol[3][rule_index]] = 1
        else:
            y[i, rule_index*4 + sol[3][rule_index]] = 1
print(X.shape, X)
print(y.shape, y)

[[6, 10, 6, 9], 264, ((2, 3), (2, 1), (1, 0)), (2, 1, 2)]
(10000, 5) [[  6.  10.   6.   9. 264.]
 [  8.   9.   8.   7. 632.]
 [  7.   4.   6.   5.   1.]
 ...
 [  2.   7.   6.  10.  54.]
 [ 10.   4.   8.   1. 121.]
 [  1.   4.   1.   8.  23.]]
(10000, 3, 4) [[[0. 0. 1. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]]

 [[0. 0. 1. 0.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]]

 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]]

 ...

 [[0. 0. 1. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]]]


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
class MultiOutputMultiClassClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, base_model=None):
        self.base_model = base_model if base_model is not None else LogisticRegression(max_iter=1000)
    
    def fit(self, X, y):
        n = y.shape[1]
        self.models = [None] * n
        for i in range(n):
            self.models[i] = self.base_model.fit(X, np.argmax(y[:, i, :], axis=1))
        
        return self
    
    def predict(self, X):
        preds = [model.predict(X) for model in self.models] # Predict each output
        return np.stack(preds, axis=1) # stack them together
    
    def predict_proba(self, X):
        probas = [model.predict_proba(X) for model in self.models]
        return np.stack(probas, axis=1)

model = MultiOutputMultiClassClassifier()
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_test)
# print(y_test)
# print(y_pred)

/home/alex/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
use_random_weights = True
def solve_with_model(x):
    # pair up two numbers 1-4, then 1-3, then 1-2.
    # For each pair, we can add, subtract, multiply or divide.
    question = [list(x[:-1]), x[-1]]
    target = question[1]
    i = 0
    y = model.predict_proba([x])[0]
    while True:
        index_order = random.choice(indices_used)
        operation_order = [random.choices(range(4), weights=y[op_index], k=1)[0] for op_index in range(3)]
        i += 1
        numbers = question[0].copy()
        valid = True
        for (index1, index2), operation_index in zip(index_order, operation_order):
            val1 = numbers[index1]
            val2 = numbers[index2]
            numbers.pop(max(index1, index2))
            numbers.pop(min(index1, index2))
            if operation_index == 3 and val2 == 0:
                valid = False
                break
            numbers.append(operations[operation_index](val1, val2))
        if valid and numbers[0] == target:
            return True, index_order, operation_order, i
    return False, None, None, -1
def print_solution(numbers, index_order, operation_order, answer):
    terms = [str(x) for x in numbers]
    for (index1, index2), operation_index in zip(index_order, operation_order):
        term1 = terms[index1]
        term2 = terms[index2]
        terms.pop(max(index1, index2))
        terms.pop(min(index1, index2))
        new_term = f"({term1} {operation_names[operation_index]} {term2})"
        terms.append(new_term)
    print(" ".join(terms), "=", answer)
debug = False
for name, X_split, y_split in [("train", X_train, y_train), ("test", X_test, y_test)]:
    print(name)
    avg_count = 0
    n_entries = 0
    for i in tqdm(range(len(X_split))):
        valid, index_order, operation_order, i = solve_with_model(X_split[i])
        if not valid:
            print("Failed to solve", question)
            break
        else:
            avg_count += i
            n_entries += 1
            # dataset.append(index_order, operation_order, i))
            if debug:
                print("input", question, "indices", index_order, "operations", operation_order, "count", i)
                print_solution(question[0], index_order, operation_order, question[1])
                print()
    print(name, "Average count", avg_count / n_entries) # 1186.1122 in order, 639.8498 random, 1376.227/1556.3125 with model (rw)

train


  0%|          | 0/8000 [00:00<?, ?it/s]

train Average count 1535.506375
test


  0%|          | 0/2000 [00:00<?, ?it/s]

test Average count 1795.912


In [ ]:
label_is_3d = True
print(solutions[0])
X = np.zeros((len(solutions), 5))
# 3 indices, 3 operations
y = np.zeros((len(solutions), 6, 4)) if label_is_3d else np.zeros((len(solutions), 3*4)) # predict the 3 operations using one hot encoding
for i, sol in enumerate(solutions):
    X[i] = sol[0] + [sol[1]] # input + target
    # y[i] = sol[3] # regression output
    # one hot encoding
    for rule_index in range(3):
        if label_is_3d:
            y[i, rule_index, sol[3][rule_index]] = 1
        else:
            y[i, rule_index*4 + sol[3][rule_index]] = 1
print(X.shape, X)
print(y.shape, y)